In [ ]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
# CHẠY TỪNG CÁI KHÔNG ĐƯỢC RUN ALL!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
import os
from PIL import Image
from torchvision import transforms
import torch
import torch.nn as nn
from torch.nn.utils import spectral_norm
import matplotlib.pyplot as plt
import numpy as np
from skimage.metrics import structural_similarity as ssim
from math import log10
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


In [ ]:
#check cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using CUDA!")
else:
    device = torch.device("cpu")
    print("GPU not available. Using CPU!")

In [ ]:
#metrics
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    return 20 * log10(1.0 / np.sqrt(mse))

def calculate_ssim(img1, img2):
    return ssim(img1, img2, multichannel=True)

In [ ]:
#utils
def save_model(model, path):
    torch.save(model.state_dict(), path)

def load_model(model, path, device):
    model.load_state_dict(torch.load(path, map_location=device,weights_only=True))
    return model

def display_images(images, titles=None):
    n = len(images)
    plt.figure(figsize=(15, 5))
    for i, img in enumerate(images):
        plt.subplot(1, n, i+1)
        plt.imshow(img.permute(1, 2, 0).cpu().numpy())
        if titles:
            plt.title(titles[i])
        plt.axis('off')
    plt.show()

In [ ]:
#datasets
class DeblurDataset(Dataset):
    def __init__(self, blur_dir, sharp_dir, transform=None):
        self.blur_dir = blur_dir
        self.sharp_dir = sharp_dir
        self.blur_images = sorted(os.listdir(blur_dir))
        self.sharp_images = sorted(os.listdir(sharp_dir))
        self.transform = transform or transforms.Compose([
            transforms.Resize((360,640)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.blur_images)

    def __getitem__(self, idx):
        blur_path = os.path.join(self.blur_dir, self.blur_images[idx])
        sharp_path = os.path.join(self.sharp_dir, self.sharp_images[idx])

        blur_img = Image.open(blur_path).convert("RGB")
        sharp_img = Image.open(sharp_path).convert("RGB")

        if self.transform:
            blur_img = self.transform(blur_img)
            sharp_img = self.transform(sharp_img)

        return blur_img, sharp_img

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self, num_residual_blocks=6):
        super(Generator, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        # Bottleneck with residual blocks
        self.bottleneck = nn.Sequential(
            *[ResidualBlock(256) for _ in range(num_residual_blocks)]
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=7, stride=1, padding=3),
            nn.Tanh()
        )

    def forward(self, x):
        # Pass through encoder
        encoded = self.encoder(x)

        # Pass through bottleneck with residual blocks
        bottleneck = self.bottleneck(encoded)

        # Decode to reconstruct the image
        decoded = self.decoder(bottleneck)
        return decoded

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            # Input: (B, 3, H, W)
            nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1),  # Downsample
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # Downsample
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # Downsample
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),  # Downsample
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0),  # Final classification
            nn.Sigmoid()  # Output: Validity score
        )

    def forward(self, img):
        validity = self.model(img)  # Output shape: (B, 1, 1, 1)
        return validity.view(-1, 1)  # Flatten to (B, 1)

In [ ]:
# Khởi tạo mô hình
generator = Generator().to(device)  # Sử dụng lớp Generator đã định nghĩa
discriminator = Discriminator().to(device)  # Sử dụng lớp Discriminator đã định nghĩa

# Dataset và DataLoader
train_dataset = DeblurDataset("/kaggle/input/gopro-deblur/gopro_deblur/blur/images", "/kaggle/input/gopro-deblur/gopro_deblur/sharp/images")
train_loader = DataLoader(train_dataset, batch_size=6, shuffle=True)
val_dataset = DeblurDataset("/kaggle/input/validation/val/blur", "/kaggle/input/validation/val/sharp")
val_loader = DataLoader(val_dataset, batch_size=6, shuffle=False)

# Hàm mất mát
adversarial_loss = nn.BCEWithLogitsLoss()  # Sử dụng BCE với logits cho đầu ra từ discriminator
pixel_loss = nn.L1Loss()  # Mất mát theo pixel (L1)

# Optimizer
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Theo dõi checkpoint tốt nhất
best_val_loss = float('inf')  # Khởi tạo giá trị rất lớn
best_checkpoint = None

# Danh sách lưu loss để vẽ biểu đồ
training_losses = []
validation_losses = []


In [ ]:
# Vòng lặp huấn luyện
num_epochs = 80
best_val_loss = float('inf')  # Giá trị khởi tạo cho loss tốt nhất
training_losses = []  # Danh sách lưu training loss
validation_losses = []  # Danh sách lưu validation loss

for epoch in range(num_epochs):
    generator.train()
    train_loss = 0  # Tổng loss cho epoch hiện tại
    for i, (blurred, sharp) in enumerate(train_loader):
        blurred, sharp = blurred.to(device), sharp.to(device)
        
        # ====== 1. Huấn luyện Discriminator ======
        
        real_output = discriminator(sharp)
        fake_images = generator(blurred)
        fake_output = discriminator(fake_images.detach())
        real_labels = torch.ones_like(real_output, device=device)
        fake_labels = torch.zeros_like(fake_output, device=device)
        d_optimizer.zero_grad()
        real_loss = adversarial_loss(real_output, real_labels)
        fake_loss = adversarial_loss(fake_output, fake_labels)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        d_optimizer.step()
        
        # ====== 2. Huấn luyện Generator ======
        
        g_optimizer.zero_grad()
        fake_output = discriminator(fake_images)
        real_labels = torch.ones_like(fake_output, device=device)
        g_adv_loss = adversarial_loss(fake_output, real_labels)
        g_pix_loss = pixel_loss(fake_images, sharp)
        g_loss = g_adv_loss + 100 * g_pix_loss
        g_loss.backward()
        g_optimizer.step()
        
        train_loss += g_loss.item()
        
        # Hiển thị log sau mỗi 10 bước
        if i % 5 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], "
                  f"D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}")

    # Tính toán và lưu training loss cho epoch hiện tại
    training_losses.append(train_loss / len(train_loader))
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {train_loss / len(train_loader):.4f}")

    # Validation sau mỗi epoch
    generator.eval()
    val_loss = 0
    with torch.no_grad():
        for blurred, sharp in val_loader:
            blurred, sharp = blurred.to(device), sharp.to(device)
            fake_images = generator(blurred)
            val_loss += pixel_loss(fake_images, sharp).item()
    
    val_loss /= len(val_loader)
    validation_losses.append(val_loss)  # Lưu validation loss cho epoch hiện tại
    print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {val_loss:.4f}")

    # Lưu checkpoint cho mỗi epoch
    checkpoint_generator = f"/kaggle/working/generatorepoch{epoch+1}.pth"
    save_model(generator, checkpoint_generator)

    # Kiểm tra và cập nhật loss tốt nhất
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_checkpoint_generator = checkpoint_generator

# Hoàn tất huấn luyện
print("Training complete!")
print(f"Best generator checkpoint: {best_checkpoint_generator} with Validation Loss: {best_val_loss:.4f}")

In [ ]:
# ===== Vẽ biểu đồ loss =====
plt.figure(figsize=(10, 6))
plt.plot(range(1, num_epochs + 1), training_losses, label="Training Loss")
plt.plot(range(1, num_epochs + 1), validation_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Validation Loss")
plt.legend()
plt.grid()
plt.savefig(f"loss_plot.png")

In [ ]:
# Load mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)  
generator = load_model(generator,"/kaggle/input/finalmodel/pytorch/default/1/generatorepoch74.pth", device)

# Tiền xử lý ảnh
transform = transforms.Compose([
    transforms.Resize((360,640)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dự đoán
img = Image.open("/kaggle/input/test-data/test/blur/310.png").convert("RGB")
input_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    output_tensor = generator(input_tensor)

# Lưu hoặc hiển thị ảnh kết quả
output_img = transforms.ToPILImage()(output_tensor.squeeze().cpu())
output_img.save("output(310).png")

In [ ]:
# Load mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator().to(device)

# Hàm load model
def load_model(model, path, device):
    state_dict = torch.load(path, map_location=device, weights_only=True)
    model.load_state_dict(state_dict)
    model.eval()
    return model

# Hàm tính PSNR
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 1.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

# Hàm tính SSIM
def calculate_ssim(img1, img2, win_size=7):
    return ssim(img1, img2, 
               channel_axis=2,
               win_size=win_size, 
               data_range=1.0)

# Load model
generator = load_model(generator, "/kaggle/input/finalmodel/pytorch/default/1/generatorepoch74.pth", device)

# Dataset và DataLoader
test_dataset = DeblurDataset("/kaggle/input/test-data/test/blur", "/kaggle/input/test-data/test/sharp")
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Biến lưu tổng PSNR, SSIM và số lượng ảnh
total_psnr = 0.0
total_ssim = 0.0
count = 0

# Đánh giá
for blurred, sharp in test_loader:
    blurred = blurred.to(device)
    sharp = sharp.to(device)
    
    # Sinh ảnh
    with torch.no_grad():
        output = generator(blurred)
    
    # Tính PSNR và SSIM
    output_resized = np.transpose(output.cpu().numpy()[0], (1, 2, 0))
    sharp_resized = np.transpose(sharp.cpu().numpy()[0], (1, 2, 0))
    output_resized = np.clip(output_resized, 0, 1)
    sharp_resized = np.clip(sharp_resized, 0, 1)
    
    psnr_val = calculate_psnr(output_resized, sharp_resized)
    min_dim = min(output_resized.shape[0], output_resized.shape[1])
    win_size = min(11, min_dim - (min_dim % 2) + 1)
    ssim_val = calculate_ssim(output_resized, sharp_resized, win_size=win_size)
    
    # Cộng dồn giá trị PSNR và SSIM
    total_psnr += psnr_val
    total_ssim += ssim_val
    count += 1
    
    # In PSNR và SSIM cho từng ảnh
    print(f"PSNR: {psnr_val:.2f}, SSIM: {ssim_val:.4f}")
    print("-" * 50)

# Tính trung bình PSNR và SSIM
avg_psnr = total_psnr / count
avg_ssim = total_ssim / count

# In trung bình PSNR và SSIM
print(f"Average PSNR: {avg_psnr:.2f}")
print(f"Average SSIM: {avg_ssim:.4f}")


In [ ]:
!pip install xlsxwriter

In [ ]:
import pandas as pd
import seaborn as sns

# Chuyển kết quả thành DataFrame
df = pd.DataFrame(results)

# Lưu dữ liệu và tạo charts trong Excel sử dụng XlsxWriter
with pd.ExcelWriter('deblur_results.xlsx', engine='xlsxwriter') as writer:
    # Sheet dữ liệu chi tiết
    df.to_excel(writer, sheet_name='Detailed Results', index=False)
    
    # Sheet thống kê
    stats = pd.DataFrame({
        'Metric': ['PSNR', 'SSIM'],
        'Mean': [df['PSNR'].mean(), df['SSIM'].mean()],
        'Std': [df['PSNR'].std(), df['SSIM'].std()],
        'Min': [df['PSNR'].min(), df['SSIM'].min()],
        'Max': [df['PSNR'].max(), df['SSIM'].max()]
    })
    stats.to_excel(writer, sheet_name='Statistics', index=False)
    
    # Lấy workbook và worksheet để tạo charts
    workbook = writer.book
    worksheet = writer.sheets['Detailed Results']
    
    # Tạo chart cho PSNR
    chart_psnr = workbook.add_chart({'type': 'line'})
    chart_psnr.add_series({
        'name': 'PSNR',
        'categories': ['Detailed Results', 1, 0, len(df), 0],
        'values': ['Detailed Results', 1, 1, len(df), 1],
        'marker': {'type': 'circle'},
    })
    chart_psnr.set_title({'name': 'PSNR Values Across Images'})
    chart_psnr.set_x_axis({'name': 'Image'})
    chart_psnr.set_y_axis({'name': 'PSNR'})
    worksheet.insert_chart('H2', chart_psnr)
    
    # Tạo chart cho SSIM
    chart_ssim = workbook.add_chart({'type': 'line'})
    chart_ssim.add_series({
        'name': 'SSIM',
        'categories': ['Detailed Results', 1, 0, len(df), 0],
        'values': ['Detailed Results', 1, 2, len(df), 2],
        'marker': {'type': 'circle'},
        'line': {'color': 'red'},
    })
    chart_ssim.set_title({'name': 'SSIM Values Across Images'})
    chart_ssim.set_x_axis({'name': 'Image'})
    chart_ssim.set_y_axis({'name': 'SSIM'})
    worksheet.insert_chart('H18', chart_ssim)

# Vẽ biểu đồ bằng matplotlib
plt.figure(figsize=(15, 10))

# PSNR plot
plt.subplot(2, 1, 1)
sns.lineplot(data=df, x='Image', y='PSNR', marker='o')
plt.title('PSNR Values Across Images')
plt.xticks(rotation=45)
plt.grid(True)

# SSIM plot
plt.subplot(2, 1, 2)
sns.lineplot(data=df, x='Image', y='SSIM', marker='o', color='red')
plt.title('SSIM Values Across Images')
plt.xticks(rotation=45)
plt.grid(True)

plt.tight_layout()
plt.savefig('metrics_plots.png', dpi=300, bbox_inches='tight')
plt.close()

# In thống kê tổng quát
print("\nThống kê tổng quát:")
print(stats.to_string(index=False))

# In giá trị trung bình
print(f"\nGiá trị trung bình:")
print(f"PSNR trung bình: {df['PSNR'].mean():.2f}")
print(f"SSIM trung bình: {df['SSIM'].mean():.4f}")